In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


# Append the analysis folder so that you can import custom packages
import sys, os
sys.path.append(r'{}'.format(os.path.abspath(os.pardir)))

# Custom packages
from wholebrain_tools import aba, dataIO
from wholebrain_tools import pvnegative as neg

from wholebrain_tools import graphics as gt


# Instantiate an Atlas object from the aba module 
# The first time you run this it will download the structures.json file from the Allen Institute server
paths = dataIO.pathParser()
structuresFile = paths.structures
A = aba.Atlas(nodes=structuresFile)
DFM = aba.AnatomyDataFrameManager(A)

# 1. Load all data

Loads data for **diffuse fluorescence** and **dots** for a **single channel** and 
creates a multi-index dataframe (df) containing raw measurements for all areas
of the brain at high resolution

In [ ]:
# --------------------------------------------------------------------
searchPath = paths.alldata
channelName = 'wfa'     # 'wfa' or 'pv'
# --------------------------------------------------------------------

df = dataIO.allMiceRegions(searchPath=searchPath, channelName=channelName, normCellIntens=True)



In [ ]:
#LOAD PV- DATA

# load the cell dataframe
df_cells = dataIO.allMiceDots(searchPath,normCellInt = True )
# exclude pv cells
df_cells = df_cells[(df_cells['wfa'] == 1)&(df_cells['pv'] == 0)]
# quantify data
dotsGroup = df_cells.groupby(by=['mouse', 'regionID']).agg(
            numCells = ('wfa','sum'),
            fluoCellSum = ('fluoMeanWfa','sum'))
#remove nans
dotsGroup['numCells'] = dotsGroup['numCells'].fillna(0)
dotsGroup = dotsGroup.reset_index()
#reshape the datafra,e
dotsGroup = dotsGroup.pivot(index='regionID',columns = 'mouse').reorder_levels(['mouse', None ] ,axis= 'columns').sort_index(axis = 1)
#name columns consistentely
dotsGroup.columns = dotsGroup.columns.set_names(['mouse', 'params'])

#replace data in the source df
df_neg = df.copy()
df_neg.update(dotsGroup)
#create multiindex
df_neg = DFM.multiIndexDf_from_fineDf(df_neg)
df_neg

In [ ]:
#LOAD total data
df_total = DFM.multiIndexDf_from_fineDf(df)
brainEnergy , brainDiffFluo =DFM.wholeBrainMetrics(df_total)

# 3. Coarse ontology bar plots
Aggregate data at coarse ontology level

In [ ]:
# Dataframe at coarse ontology
total_coarse = DFM.regionsDf_to_coarse(df_total, normalize=True)
coarse_neg = neg.subsetDf_to_coarse(df_neg,A, normEnergy=brainEnergy, normDiffuse=brainDiffFluo ,normalize=True)


## 3.2 Barplot - PNN Energy

In [ ]:
energy = coarse_neg.xs('energy', axis=1, level='params')
energy_tot = total_coarse.xs('energy', axis=1, level='params')
# Plot the data
g = gt.coarseOntologyBarplot(energy, A, 
    title='Whole Brain',#r'PV$^{-}$ PNN Energy',
    cmap='OrRd',
    xlabel='Fold Enrichment',
    areaNames=True,
    )

data = energy_tot.melt(ignore_index=False).reset_index()
g =sns.barplot(
    data, 
    x =  'value', 
    y = A.ids_to_names(data['coarse']), 
    zorder = 0, 
    errorbar=None,
    edgecolor = 'lightgray',
    linewidth=.5,
    color='lightgray',
    ax=g
)


# maxen = energy_tot.mean(axis='columns').max()

g.set_xlabel(r'PV$^{-}$ PNN Energy (A.U.)')
savepath = os.getcwd()
# plt.savefig(os.path.join(savepath,'pvneg_atlas', f"coarse_areas.pdf"), bbox_inches="tight")



In [ ]:
total_mid = DFM.regionsDf_to_mid(df_total, normalize=True)
neg_mid = neg.subsetDf_to_mid(df_neg,A, normEnergy=brainEnergy, normDiffuse=brainDiffFluo ,normalize=True)

In [ ]:
energy_mid = total_mid.xs('energy', axis='columns', level='params')
energy_neg_mid = neg_mid.xs('energy', axis='columns', level='params')

In [ ]:
for areaid in A.get_major_divisions_ids():
    gt.energySubsetBarplot(neg_mid.loc[areaid], A, x='energy',
                                    figsize=(6,6), cmap='OrRd',
                                    xlabel=r'PV$^{-}$ PNN Energy (A.U.)', title = A.ids_to_names([areaid])[0],
                                    fontScaling = 1, adaptiveHeight=True, dots=False)

    ax = plt.gca()
    meltedData = energy_mid.loc[areaid].melt(ignore_index=False)
    sns.barplot(
                    data=meltedData.dropna(),
                    ax=ax,
                    y=A.ids_to_names(meltedData.dropna().index.tolist()),
                    x='value',
                    orient="h",
                    alpha=1,
                    linewidth=.5,
                    edgecolor="lightgray",
                    color='lightgray',
                    errorbar=None,zorder = 0,
                )

    plt.xlabel(r'PV$^{-}$ PNN Energy (A.U.)')
    # plt.savefig(os.path.join(savepath,'pvneg_atlas', f"energy_diff_mid_{A.ids_to_acronyms([areaid])[0]}.svg"), bbox_inches="tight")

